In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [5]:
movies_df = pd.read_csv('movies-2.csv',usecols=['movieId','title'],dtype={'movieId': 'int32', 'title': 'str'})
rating_df = pd.read_csv('ratings.csv', usecols=['userId', 'movieId', 'rating'],
    dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

In [6]:
movies_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [7]:
rating_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [8]:
df = pd.merge(rating_df, movies_df, on = 'movieId')
df.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [9]:
df.shape

(100836, 4)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   100836 non-null  int32  
 1   movieId  100836 non-null  int32  
 2   rating   100836 non-null  float32
 3   title    100836 non-null  object 
dtypes: float32(1), int32(2), object(1)
memory usage: 1.9+ MB


In [12]:
df.duplicated().sum()

0

In [13]:
movie_rating_count = (df.groupby(['title'])['rating'].count().reset_index().
                      rename(columns = {'rating': 'TotalRatingCount'})
                      [['title', 'TotalRatingCount']])


movie_rating_count.head()

                                                                                             

,title,TotalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [33]:
movie_rating_count.shape

(9719, 2)

In [16]:
movie_with_RatingCount = df.merge(movie_rating_count, left_on = 'title', 
                                  right_on = 'title', how = 'left')


movie_with_RatingCount.head()                          

,userId,movieId,rating,title,TotalRatingCount
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [19]:
## Checking TotalRatingCount column

pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(movie_with_RatingCount['TotalRatingCount'].describe())

count   100836.000
mean        58.759
std         61.965
min          1.000
25%         13.000
50%         39.000
75%         84.000
max        329.000
Name: TotalRatingCount, dtype: float64


In [23]:
pop_threshold = 50

rating_popular_movie = movie_with_RatingCount.query('TotalRatingCount >= @pop_threshold')
rating_popular_movie.head()

,userId,movieId,rating,title,TotalRatingCount
0,1,1,4.000,Toy Story (1995),215
1,5,1,4.000,Toy Story (1995),215
2,7,1,4.500,Toy Story (1995),215
3,15,1,2.500,Toy Story (1995),215
4,17,1,4.500,Toy Story (1995),215


In [24]:
rating_popular_movie.shape

(41362, 5)

In [30]:
title_with_pop_ratingcount = rating_popular_movie.groupby(['title'])['TotalRatingCount'].count().sort_values(ascending = False).reset_index()

In [32]:
title_with_pop_ratingcount.head(15)

,title,TotalRatingCount
0,Forrest Gump (1994),329
1,"Shawshank Redemption, The (1994)",317
2,Pulp Fiction (1994),307
3,"Silence of the Lambs, The (1991)",279
4,"Matrix, The (1999)",278
5,Star Wars: Episode IV - A New Hope (1977),251
6,Jurassic Park (1993),238
7,Braveheart (1995),237
8,Terminator 2: Judgment Day (1991),224
9,Schindler's List (1993),220


In [36]:
## Creting a Pivot table

movies_features_df = rating_popular_movie.pivot_table(index = 'title', columns= 'userId', values= 'rating').fillna(0)

movies_features_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,3.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000
12 Angry Men (1957),0.000,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,...,5.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2001: A Space Odyssey (1968),0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,5.000,0.000,3.000,0.000,4.500
28 Days Later (2002),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.500,0.000,5.000
300 (2007),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,...,0.000,0.000,0.000,0.000,3.000,0.000,0.000,5.000,0.000,4.000


In [39]:
from scipy.sparse import csr_matrix

movies_features_df_matrix = csr_matrix(movies_features_df.values)

from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movies_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [40]:
movies_features_df.shape

(450, 606)

In [61]:
query_index = np.random.choice(movies_features_df.shape[0])
print(query_index)

distances, indices = model_knn.kneighbors(movies_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)
distances, indices


125


(array([[1.1920929e-07, 3.3684438e-01, 3.5694325e-01, 3.7534332e-01,
         3.8428169e-01, 4.0701991e-01]], dtype=float32),
 array([[125, 215, 395, 126, 385, 386]]))

In [55]:
movies_features_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,3.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000
12 Angry Men (1957),0.000,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,...,5.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2001: A Space Odyssey (1968),0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,5.000,0.000,3.000,0.000,4.500
28 Days Later (2002),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.500,0.000,5.000
300 (2007),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,...,0.000,0.000,0.000,0.000,3.000,0.000,0.000,5.000,0.000,4.000


In [62]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(movies_features_df.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}'.format(i, movies_features_df.index[indices.flatten()[i]], distances.flatten()[i]))
        

Recommendations for Die Hard (1988):

1: Indiana Jones and the Last Crusade (1989), with distance of 0.33684438467025757
2: Terminator, The (1984), with distance of 0.3569432497024536
3: Die Hard 2 (1990), with distance of 0.37534332275390625
4: Star Wars: Episode V - The Empire Strikes Back (1980), with distance of 0.3842816948890686
5: Star Wars: Episode VI - Return of the Jedi (1983), with distance of 0.4070199131965637
